In [5]:
%matplotlib inline
import menpo.io as mio
from menpo.shape import PointCloud
from menpofit.modelinstance import OrthoPDM
import numpy as np
import pickle

left_brow_inds = np.arange(47, 52)
right_brow_inds = np.arange(52, 57)
left_eye_inds = np.arange(27, 37) # np.append(np.arange(27, 37), 57)
right_eye_inds = np.arange(37, 47) # np.append(np.arange(37, 47), 58)
jaw_line_inds = np.arange(0, 17)
mouth_inds = np.arange(17, 27)

training_shapes = []
r_eyebrow_train_shapes = []
l_eyebrow_train_shapes = []
r_eye_train_shapes = []
l_eye_train_shapes = []
chin_train_shapes = []
mouth_train_shapes = []
temp = mio.import_landmark_files('../data/images/Manga_Cropped_60_Points_v2/train/*.pts', verbose=True)
for l in temp:
    pts = l
    training_shapes.append(pts)
    r_eyebrow_train_shapes.append(PointCloud(pts.points[right_brow_inds]))
    l_eyebrow_train_shapes.append(PointCloud(pts.points[left_brow_inds]))
    r_eye_train_shapes.append(PointCloud(pts.points[right_eye_inds]))
    l_eye_train_shapes.append(PointCloud(pts.points[left_eye_inds]))
    chin_train_shapes.append(PointCloud(pts.points[jaw_line_inds]))
    mouth_train_shapes.append(PointCloud(pts.points[mouth_inds]))

#r_eyebrow_train_shapes[0]
#l_eyebrow_train_shapes[0]
#r_eye_train_shapes[0]
#l_eye_train_shapes[0]
#chin_train_shapes[0]
#mouth_train_shapes[0]
#pt = PointCloud(l_eyebrow_train_shapes)
#pt.view()

r_eyebrow_shape_model = OrthoPDM(r_eyebrow_train_shapes, max_n_components=None)
l_eyebrow_shape_model = OrthoPDM(l_eyebrow_train_shapes, max_n_components=None)
r_eye_shape_model = OrthoPDM(r_eye_train_shapes, max_n_components=None)
l_eye_shape_model = OrthoPDM(l_eye_train_shapes, max_n_components=None)
chin_shape_model = OrthoPDM(chin_train_shapes, max_n_components=None)
mouth_shape_model = OrthoPDM(mouth_train_shapes, max_n_components=None)
all_shape_model = OrthoPDM(training_shapes, max_n_components=None)
'''
r_eyebrow_shape_model.n_active_components = 0.9
l_eyebrow_shape_model.n_active_components = 0.9
r_eye_shape_model.n_active_components = 0.9
l_eye_shape_model.n_active_components = 0.9
chin_shape_model.n_active_components = 0.9
mouth_shape_model.n_active_components = 0.9
'''

r_brow_pc_list = [2, 3, 4, 5, 6] # max 6
l_brow_pc_list = [2, 3, 4, 5, 6] # max 6
r_eye_pc_list = [2, 3, 4, 6, 8, 12, 16] # max 8                     [2, 3, 4, 6, 8] -> 16
l_eye_pc_list = [2, 3, 4, 6, 8, 12, 16] # max 8                    -> 16
chin_pc_list = [5, 7, 10, 16, 20] # max 30
mouth_pc_list = [2, 3, 4, 6, 8, 12, 16] # max 36                   -> 16
all_pc_list = [30]

parts = ['r_brow', 'l_brow', 'r_eye', 'l_eye', 'chin', 'mouth', 'all']
part_pc_lists = [r_brow_pc_list, l_brow_pc_list, r_eye_pc_list, l_eye_pc_list,
                chin_pc_list, mouth_pc_list, all_pc_list]
part_models = [r_eyebrow_shape_model, l_eyebrow_shape_model, r_eye_shape_model, 
               l_eye_shape_model, chin_shape_model, mouth_shape_model, all_shape_model]

for i, part in enumerate(parts):
    part_pc_list = part_pc_lists[i]
    part_model = part_models[i]
    for j in range(len(part_pc_list)):
        part_model.n_active_components = part_pc_list[j]
        filehandler = open('../data/pdm_models/' + part + '_' + str(part_pc_list[j]), "wb")
        pickle.dump(part_model, filehandler)
        filehandler.close()
'''      
print(r_eyebrow_shape_model)
print(l_eyebrow_shape_model)
print(r_eye_shape_model)
print(l_eye_shape_model)
print(chin_shape_model)
print(mouth_shape_model)
'''

print('done')

Found 1151 assets, index the returned LazyList to import.
done


In [ ]:
def pdm_correct(init_shape, pdm_model, part_inds=None):
    """ correct landmarks using pdm (point distribution model)"""
    pdm_model.set_target(PointCloud(init_shape))
    if part_inds is None:
        return pdm_model.target.points
    else:
        return pdm_model.target.points[part_inds]

In [ ]:
new_lms = np.zeros((60, 2))
shape = mio.import_landmark_file('../data/images/Manga_Cropped_60_Points_v2/train/MeteoSanStrikeDesu_032_face00_x394y69w257h266.pts')
#shape = shape['PTS']
lms_init = shape.points
parts = ['l_brow', 'r_brow', 'l_eye', 'r_eye', 'mouth', 'jaw']
part_inds_opt = [left_brow_inds, right_brow_inds, left_eye_inds, right_eye_inds, mouth_inds, jaw_line_inds]
models = [l_eyebrow_shape_model, r_eyebrow_shape_model, l_eye_shape_model, r_eye_shape_model, mouth_shape_model, chin_shape_model]

for i, part in enumerate(parts):
    part_inds = part_inds_opt[i]
    pdm_temp = models[i]
    #pc = pc_opt[i]
    '''
    temp_model = os.path.join(models_dir, train_type + '_' + part + '_' + str(pc))
    filehandler = open(temp_model, "rb")
    try:
        pdm_temp = pickle.load(filehandler)
    except UnicodeDecodeError:
        pdm_temp = pickle.load(filehandler, fix_imports=True, encoding="latin1")
    filehandler.close()
    '''

    #if patches is None:
    part_lms_pdm = pdm_correct(lms_init[part_inds], pdm_temp)
    #else:
        #part_lms_pdm = w_pdm_correct(
            #init_shape=lms_init[part_inds], patches=patches, pdm_model=pdm_temp, part_inds=part_inds)

    new_lms[part_inds] = part_lms_pdm
new_lms[59] = lms_init[59]
new_lms

new_shape = PointCloud(new_lms)
print(new_shape.points)

In [10]:

filehandler = open('../data/pdm_models/chin_20', "rb")
model = pickle.load(filehandler)
model.weights

array([  3.00173302e-15,   3.75221035e-15,   3.74528154e-15,
         5.50079693e-15,  -5.19941821e-15,  -2.01153290e-15,
        -1.04121696e-15,  -8.65542521e-15,  -1.04926013e-16,
        -2.41269242e-15,   1.36947366e-15,   3.65449618e-15,
        -1.15486236e-15,  -1.05230471e-15,  -5.07598729e-15,
         3.61036173e-15,  -2.25498159e-15,  -4.27173747e-15,
        -2.95888465e-15,  -5.35056150e-15])